In [59]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error as mape
from dateutil.relativedelta import relativedelta

nb_name = '001'

# Process section

In [51]:
def feature_engineering(df, **kwargs):
    """
    Feature Engineering
    """

    # Validation
    if 'date' not in df.columns:
        df['date'] = df.index
        # raise ValueError(f'date column is not in the dataframe. It has {df.columns}')
    def _fill_params(_prodf):
        _prodf['date'] = pd.to_datetime(_prodf['date'])
        _prodf['month'] = _prodf['date'].dt.month
        _prodf['day'] = _prodf['date'].dt.day
        _prodf['year'] = _prodf['date'].dt.year
        _prodf['weekday'] = _prodf['date'].dt.weekday
        _prodf['week'] = _prodf['date'].dt.isocalendar().week
        _prodf['quarter'] = _prodf['date'].dt.quarter
        _prodf['is_month_start'] = _prodf['date'].dt.is_month_start
        _prodf['is_month_end'] = _prodf['date'].dt.is_month_end
        _prodf['is_quarter_start'] = _prodf['date'].dt.is_quarter_start
        _prodf['is_quarter_end'] = _prodf['date'].dt.is_quarter_end
        _prodf['is_year_start'] = _prodf['date'].dt.is_year_start
        _prodf['is_year_end'] = _prodf['date'].dt.is_year_end
        #True:1, False:0 TODO: Apply one hot encoding
        _prodf['is_month_start'] = _prodf['is_month_start'].astype(int)
        _prodf['is_month_end'] = _prodf['is_month_end'].astype(int)
        _prodf['is_quarter_start'] = _prodf['is_quarter_start'].astype(int)
        _prodf['is_quarter_end'] = _prodf['is_quarter_end'].astype(int)
        _prodf['is_year_start'] = _prodf['is_year_start'].astype(int)
        _prodf['is_year_end'] = _prodf['is_year_end'].astype(int)
        _prodf['is_weekend'] = _prodf['weekday'].apply(lambda x: 1 if x >= 5 else 0)
        _prodf['is_weekday'] = _prodf['weekday'].apply(lambda x: 1 if x < 5 else 0)
        return _prodf
    
    def _fill_nan_with_10day_mean(_df_with_nan, _df_without_nan):
        _10day_mean_df = pd.DataFrame(_df_without_nan['num_sold'].resample('10D').mean())
        for idx in _10day_mean_df.index:
            mask = (_df_with_nan.index >= idx) & (_df_with_nan.index <= idx + relativedelta(days=10))
            _df_with_nan.loc[mask, 'num_sold'] = _10day_mean_df.loc[idx, 'num_sold']
        return _df_with_nan
    has_nan = df.isna().any(axis=1)
    _df_with_nan = df[has_nan]
    _df_with_nan = _df_with_nan.copy()
    _df_without_nan = df[~has_nan].copy()
    _df = _fill_params(_df)
    _df_filled = _fill_nan_with_10day_mean(_df_with_nan, _df_without_nan)
    _df_filled = _fill_params(_df_filled)
    _df = pd.concat([_df_without_nan, _df_filled], axis=0)
    _df = _df.sort_index()

    # label encoding
    for _col in ['store', 'product', 'country']:
        _enc = OneHotEncoder(sparse_output=False)
        _enc.fit(_df[_col].values.reshape(-1, 1))
        _df_enc_arr = _enc.transform(_df[_col].values.reshape(-1, 1))
        _df_enc = pd.DataFrame(_df_enc_arr, columns=_enc.categories_)
        _df_enc.index = _df.index
        _df = pd.concat([_df, _df_enc], axis=1)
        _df.drop(_col, axis=1, inplace=True)
    
    # All columns are numeric
    for _col in _df.columns:
        if _df[_col].dtype == 'object':
            _df[_col] = _df[_col].astype(float)
        elif _df[_col].dtype == 'bool':
            _df[_col] = _df[_col].astype(int)
        elif _df[_col].dtype == 'datetime64[ns]':
            _df[_col] = _df[_col].astype(int)
        if type(_col) == tuple:
            _df.rename(columns={_col: str(_col[0])}, inplace=True)
    return _df

data_path = '../datas/train.csv'

train_df = pd.read_csv(data_path, parse_dates=True, index_col='date')
modified_train_df = feature_engineering(train_df)
display(modified_train_df.head())

,id,num_sold,month,day,year,weekday,week,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_weekend,is_weekday,num_sold_10day_mean,num_sold_10day_sum,num_sold_10day_std,num_sold_10day_max,num_sold_10day_min,num_sold_week_mean,num_sold_week_sum,num_sold_week_std,num_sold_week_max,num_sold_week_min,num_sold_month_mean,num_sold_month_sum,num_sold_month_std,num_sold_month_max,num_sold_month_min,num_sold_year_mean,num_sold_year_sum,num_sold_year_std,num_sold_year_max,num_sold_year_min,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode,Canada,Finland,Italy,Kenya,Norway,Singapore
0,1,973.0,1,1,2010,4,53,1,1,0,1,0,1,0,0,1,688.409568,619568.611435,589.332315,3276.0,5.0,968.077901,261381.033295,865.380186,4541.0,5.0,737.916909,2.058788e+06,648.600672,4541.0,5.0,710.639321,2.334450e+07,613.764953,4541.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,906.0,1,1,2010,4,53,1,1,0,1,0,1,0,0,1,688.409568,619568.611435,589.332315,3276.0,5.0,968.077901,261381.033295,865.380186,4541.0,5.0,737.916909,2.058788e+06,648.600672,4541.0,5.0,710.639321,2.334450e+07,613.764953,4541.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,423.0,1,1,2010,4,53,1,1,0,1,0,1,0,0,1,688.409568,619568.611435,589.332315,3276.0,5.0,968.077901,261381.033295,865.380186,4541.0,5.0,737.916909,2.058788e+06,648.600672,4541.0,5.0,710.639321,2.334450e+07,613.764953,4541.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,491.0,1,1,2010,4,53,1,1,0,1,0,1,0,0,1,688.409568,619568.611435,589.332315,3276.0,5.0,968.077901,261381.033295,865.380186,4541.0,5.0,737.916909,2.058788e+06,648.600672,4541.0,5.0,710.639321,2.334450e+07,613.764953,4541.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,300.0,1,1,2010,4,53,1,1,0,1,0,1,0,0,1,688.409568,619568.611435,589.332315,3276.0,5.0,968.077901,261381.033295,865.380186,4541.0,5.0,737.916909,2.058788e+06,648.600672,4541.0,5.0,710.639321,2.334450e+07,613.764953,4541.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# Nan check
with open(f'../docs/{nb_name}_nan_check.txt', 'w') as f:
    # Nan check
    f.write(f'Nan check\n')
    f.write(f'{modified_train_df.isna().sum()}\n')
modified_train_df1 = modified_train_df.dropna()
modified_train_df1.to_csv(f'../modified_data/{nb_name}_train.csv', index=False)

In [53]:
print(modified_train_df1.head())
for _col in modified_train_df1.columns:
    print(f'{_col}: {modified_train_df1[_col].dtype}')

   id  num_sold  month  day  year  ...  Finland  Italy  Kenya  Norway  Singapore
0   1     973.0      1    1  2010  ...      0.0    0.0    0.0     0.0        0.0
1   2     906.0      1    1  2010  ...      0.0    0.0    0.0     0.0        0.0
2   3     423.0      1    1  2010  ...      0.0    0.0    0.0     0.0        0.0
3   4     491.0      1    1  2010  ...      0.0    0.0    0.0     0.0        0.0
4   5     300.0      1    1  2010  ...      0.0    0.0    0.0     0.0        0.0

[5 rows x 50 columns]
id: int64
num_sold: float64
month: int32
day: int32
year: int32
weekday: int32
week: UInt32
quarter: int32
is_month_start: int64
is_month_end: int64
is_quarter_start: int64
is_quarter_end: int64
is_year_start: int64
is_year_end: int64
is_weekend: int64
is_weekday: int64
num_sold_10day_mean: float64
num_sold_10day_sum: float64
num_sold_10day_std: float64
num_sold_10day_max: float64
num_sold_10day_min: float64
num_sold_week_mean: float64
num_sold_week_sum: float64
num_sold_week_std: float

In [60]:
# Learning
test_path = '../datas/test.csv'
test_df = pd.read_csv(test_path, parse_dates=True, index_col='date')
clf = RandomForestRegressor(random_state=1234)
target = 'num_sold'
y_train = modified_train_df1[target]
X_train = modified_train_df1.drop(target, axis=1)
clf.fit(X_train, y_train)

# Prediction
modified_test_df = feature_engineering(test_df)
modified_test_df1 = modified_test_df.dropna()
modified_test_df1.to_csv(f'../modified_data/{nb_name}_test.csv', index=False)
pred = clf.predict(modified_test_df.drop(target, axis=1))
modified_test_df['num_sold_pred'] = pred

# Evaluation
y_true = modified_test_df[target]
y_pred = modified_test_df['num_sold_pred']
mape_val = mape(y_true, y_pred)
print(f'MAPE: {mape_val}')

KeyError: 'num_sold'

# Trial Session

## データの理解
EDA

In [34]:
data_path = '../datas/train.csv'
test_df = pd.read_csv(data_path, parse_dates=True, index_col='date')

In [17]:
datas = ['test', 'train', 'sample_submission']
for data in datas:
    data_path = f'../datas/{data}.csv'
    _data_doc = f'../docs/{nb_name}_{data}_doc.txt'
    _df = pd.read_csv(data_path)
    with open(_data_doc, 'w') as f:
        f.write('Data Description\n')
        f.write('----------\n')
        f.write(f'{data} data has {_df.shape} and {len(_df.columns)} columns.\n')
        f.write(f'{data} data columns: {_df.columns}\n')
        f.write(f'{data} data head: {_df.head()}\n')
        f.write('----------\n')
        for col in _df.columns:
            f.write(f'{col} has {_df[col].unique()}.\n')
            f.write(f'{col} has {_df[col].nunique()} unique values.\n')
            f.write('----------\n')

In [26]:
# 欠損値処理
nan_cols = test_df.columns[test_df.isnull().any()]
nan_index = test_df.index[test_df.loc[:, nan_cols].isnull().any(axis=1)]
print(f'nan columns: {nan_cols}')
print(f'nan index: {nan_index}, {len(nan_index)}')

nan columns: Index(['num_sold'], dtype='object')
nan index: Index([     0,     45,     90,    135,    180,    225,    270,    315,    320,
          360,
       ...
       229680, 229725, 229770, 229815, 229860, 229905, 229950, 229995, 230040,
       230085],
      dtype='int64', length=8871), 8871


In [90]:
test_df1 = test_df.copy()
# mean_df1 = pd.DataFrame(test_df1['num_sold'].resample('10D').mean())
# mean_df1['to_dt'] = mean_df1.index
# for idx in mean_df1.index:
#     mean_df1.loc[idx, 'from_dt'] = mean_df1.loc[idx, 'to_dt'] - relativedelta(days=10)
#     mask = (test_df1.index > mean_df1.loc[idx, 'from_dt']) & (test_df1.index <= mean_df1.loc[idx, 'to_dt'])
#     test_df1.loc[mask, 'num_sold_10day'] = mean_df1.loc[idx, 'num_sold']

intervals = [['10D', 10],['YE', 365]]
for interval in intervals:
    mean_df = pd.DataFrame(test_df['num_sold'].resample(interval[0]).mean())
    mean_df['to_dt'] = mean_df.index
    for idx in mean_df.index:
        mean_df.loc[idx, 'from_dt'] = mean_df.loc[idx, 'to_dt'] - relativedelta(days=interval[1])
        mask = (test_df1.index > mean_df.loc[idx, 'from_dt']) & (test_df1.index <= mean_df.loc[idx, 'to_dt'])
        test_df1.loc[mask, f'num_sold_{interval[0]}'] = mean_df.loc[idx, 'num_sold']

display(test_df1.tail())

# mean_df2 = pd.DataFrame(test_df['num_sold'].resample('YE').mean())
# mean_df2['to_dt'] = mean_df2.index
# for idx in mean_df2.index:
#     mean_df2.loc[idx, 'from_dt_day'] = mean_df2.loc[idx, 'to_dt'] - relativedelta(days=365)
#     mask = (test_df1.index > mean_df2.loc[idx, 'from_dt_day']) & (test_df1.index <= mean_df2.loc[idx, 'to_dt'])
#     test_df1.loc[mask, 'num_sold_year'] = mean_df2.loc[idx, 'num_sold']



,id,country,store,product,num_sold,month,day,year,weekday,week,...,num_sold_ME_max,num_sold_ME_min,num_sold_YE_mean,num_sold_YE_sum,num_sold_YE_median,num_sold_YE_std,num_sold_YE_max,num_sold_YE_min,num_sold_10D,num_sold_YE
date,,,,,,,,,,,,,,,,,,,,,
2016-12-31,230125,Singapore,Premium Sticker Mart,Holographic Goose,466.0,12,31,2016,5,52,...,3465.0,5.0,677.567066,21524273.0,556.0,587.110884,3466.0,5.0,NaN,677.567066
2016-12-31,230126,Singapore,Premium Sticker Mart,Kaggle,2907.0,12,31,2016,5,52,...,3465.0,5.0,677.567066,21524273.0,556.0,587.110884,3466.0,5.0,NaN,677.567066
2016-12-31,230127,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0,12,31,2016,5,52,...,3465.0,5.0,677.567066,21524273.0,556.0,587.110884,3466.0,5.0,NaN,677.567066
2016-12-31,230128,Singapore,Premium Sticker Mart,Kerneler,1242.0,12,31,2016,5,52,...,3465.0,5.0,677.567066,21524273.0,556.0,587.110884,3466.0,5.0,NaN,677.567066
2016-12-31,230129,Singapore,Premium Sticker Mart,Kerneler Dark Mode,1622.0,12,31,2016,5,52,...,3465.0,5.0,677.567066,21524273.0,556.0,587.110884,3466.0,5.0,NaN,677.567066


In [48]:
# print(test_df['num_sold'].resample('10D').mean())
# print(test_df['num_sold'].resample('YE').sum())
intervals = ['D', '10D','W', 'ME', 'YE']
fig_fol = f'../results/'

for i, interval in enumerate(intervals):
    fig, axes = plt.subplots(len(test_df['store'].unique()), 1,figsize=(7, 5))
    for j, store in enumerate(test_df['store'].unique()): # storeごとの売り上げを確認
        for k, product in enumerate(test_df['product'].unique()):
            _plot_df = test_df[(test_df['store'] == store) & (test_df['product'] == product)]
            _means = _plot_df['num_sold'].resample(interval).mean()
            axes[j].plot(_means, label=product)
        axes[j].set_title(f'{store}')
    fig.suptitle(f'num_sold resample by {str(interval)}')
    plt.tight_layout()
    plt.legend()
    plt.savefig(f'{fig_fol}/{nb_name}_num_sold_resample_{interval}.png')
    # plt.show()
    plt.close()
    

,index,id,country,store,product,num_sold,month,day,year,weekday,...,num_sold_month_mean,num_sold_month_sum,num_sold_month_std,num_sold_month_max,num_sold_month_min,num_sold_year_mean,num_sold_year_sum,num_sold_year_std,num_sold_year_max,num_sold_year_min
0,0,0,Canada,Discount Stickers,Holographic Goose,NaN,1,1,2010,4,...,738.456075,1975370.0,662.290537,4541.0,5.0,710.779776,22134393.0,630.348568,4541.0,5.0
1,1,1,Canada,Discount Stickers,Kaggle,973.0,1,1,2010,4,...,738.456075,1975370.0,662.290537,4541.0,5.0,710.779776,22134393.0,630.348568,4541.0,5.0
2,2,2,Canada,Discount Stickers,Kaggle Tiers,906.0,1,1,2010,4,...,738.456075,1975370.0,662.290537,4541.0,5.0,710.779776,22134393.0,630.348568,4541.0,5.0
3,3,3,Canada,Discount Stickers,Kerneler,423.0,1,1,2010,4,...,738.456075,1975370.0,662.290537,4541.0,5.0,710.779776,22134393.0,630.348568,4541.0,5.0
4,4,4,Canada,Discount Stickers,Kerneler Dark Mode,491.0,1,1,2010,4,...,738.456075,1975370.0,662.290537,4541.0,5.0,710.779776,22134393.0,630.348568,4541.0,5.0


/Users/teramoto/.pyenv/versions/miniforge3-23.11.0-0/envs/datasci/lib/python3.13/site-packages/pandas/plotting/_matplotlib/misc.py:100: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.set_xlim(boundaries_list[j])
/Users/teramoto/.pyenv/versions/miniforge3-23.11.0-0/envs/datasci/lib/python3.13/site-packages/pandas/plotting/_matplotlib/misc.py:101: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(boundaries_list[i])
/Users/teramoto/.pyenv/versions/miniforge3-23.11.0-0/envs/datasci/lib/python3.13/site-packages/pandas/plotting/_matplotlib/misc.py:91: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.set_xlim(boundaries_list[i])


Error in callback <function _draw_all_if_interactive at 0x14e04ec00> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

## Encoding test

In [ ]:
data_path = '../datas/train.csv'
test_df = pd.read_csv(data_path, parse_dates=True, index_col='date')
for item in ['product', 'store']:
    enc = OneHotEncoder(sparse_output=False)
    enc.fit(test_df[item].values.reshape(-1, 1))
    encoded = enc.transform(test_df[[item]].values.reshape(-1, 1))
    df_encoded = pd.DataFrame(encoded, columns=enc.categories_)
    df_encoded.index = test_df.index
    display(df_encoded.head())
    display(df_encoded.shape)
    display(test_df.shape)
    test_df = pd.concat([test_df, df_encoded], axis=1)
display(test_df.head())